In [ ]:
from matplotlib import pyplot, cm
from math import pi
import numpy
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [ ]:
nx = 41
ny = 41

l = 1.
h = 1.

dx = l/(nx-1)
dy = h/(ny-1)

x = numpy.linspace(0,1,nx)
y = numpy.linspace(0,1,ny)

l1_target = 1e-6

In [ ]:
def L1norm(new, old):
    norm = numpy.sum(numpy.abs(new-old))
    return norm

In [ ]:
omega = numpy.zeros((ny,nx)) 
psi = numpy.zeros((ny,nx)) 

In [ ]:
def cavity(omega, psi, dx, dy, ll_target):
    l1norm = 1
    omega_n = numpy.empty_like(omega)
    psi_n = numpy.empty_like(psi)
    
    
    while l1norm > l1_target:
        omega_n = omega.copy()
        psi_n = psi.copy()
        
        omega[1:-1,1:-1] = .25 * (omega_n[1:-1,2:] + omega_n[1:-1, :-2] + omega_n[2:, 1:-1] + omega_n[:-2, 1:-1])
        psi[1:-1,1:-1] = 1./(2*(dx**2 + dy**2)) * ((psi_n[1:-1,2:]+psi_n[1:-1,:-2])*dy**2 +\
                        (psi_n[2:,1:-1] + psi_n[:-2,1:-1])*dx**2 + omega[1:-1,1:-1]*dx**2*dy**2)
        uj =1
        #boundary
        omega[-1, 1:-1] = (-1./(2.*dy**2)*((8.*psi[-2, 1:-1]) - psi[-3, 1:-1])) - ((3. * uj) / dy)
        omega[0, 1:-1] = (-1./(2.*dy**2)*((8.*psi[1, 1:-1]) - psi[2, 1:-1]))
        omega[1:-1, 0] = (-1./(2.*dx**2)*((8.*psi[1:-1, 1]) - psi[1:-1, 2]))
        omega[1:-1, -1] = (-1./(2.*dx**2)*((8.*psi[1:-1, -2]) - psi[1:-1, -3]))
      
        
        
        l1norm = max(L1norm(omega, omega_n), L1norm(psi, psi_n))
    return omega, psi

In [ ]:
A = cavity(omega.copy(), psi.copy(), dx, dy, l1_target)

In [ ]:
psi=A[1]

In [ ]:
maxpsi = numpy.absolute(psi).max()

In [ ]:
maxpsi


In [ ]:
numpy.round(psi[32,::8], 4)